In [1]:
import csv
import numpy as np
import matplotlib.pyplot as plt
from numpy import loadtxt

In [2]:
bus = 73
Bus_data = loadtxt('BusData.dat')
Gen_data = loadtxt('GenData.dat')
Bra_data = loadtxt('BranchData.dat')

In [3]:
## Randomize load data between -/+ 10%
def rand_load(bus,data):
    random = (np.random.rand(bus) * 2 - 1)*0.1
    load = data[:,2] * (1 + random)
    return load

In [4]:
from pyomo.environ import *


# instance the pyomo abstract model
model = AbstractModel()

### set
model.BUS    = Set(initialize = Bus_data[:,0]-1)                        #k
model.BRANCH = Set(initialize = Bra_data[:,0]-1)                        #j
model.GEN    = Set(initialize = Gen_data[:,0]-1)                        #i

### Param
#Bus param = k
model.bus_num  = Param(model.BUS, initialize = Bus_data[:,1]-1)


# load_profile = rand_load(bus,Bus_data)
# def bus_Pd(model, i):
#     return load_profile[int(i-1)]
# model.bus_Pd   = Param(model.BUS, initialize = bus_Pd, mutable = True)

load_profile = rand_load(bus,Bus_data)
def bus_Pd(model, k):
    return load_profile[int(k)]
model.bus_Pd   = Param(model.BUS, initialize = bus_Pd)#, mutable = True)

model.bus_type = Param(model.BUS, initialize = Bus_data[:,3])


#Gen param = i
model.gen_num  = Param(model.GEN, initialize = Gen_data[:,1]-1)

model.gen_bus  = Param(model.GEN, initialize = Gen_data[:,2]-1)

model.gen_Pmax = Param(model.GEN, initialize = Gen_data[:,3])#, mutable=True)

model.gen_Pmin = Param(model.GEN, initialize = Gen_data[:,4])#, mutable=True)

model.gen_C    = Param(model.GEN, initialize = Gen_data[:,5])



#Branch param = j
model.branch_num   = Param(model.BRANCH, initialize = Bra_data[:,1]-1)

model.branch_fbus  = Param(model.BRANCH, initialize = Bra_data[:,2]-1)

model.branch_tbus  = Param(model.BRANCH, initialize = Bra_data[:,3]-1)

model.branch_x     = Param(model.BRANCH, initialize = Bra_data[:,4])

model.branch_rateA = Param(model.BRANCH, initialize = Bra_data[:,5])#, mutable=True)

#base MVA
model.Base = Param(default=100.0)

### Var
model.bus_angle  = Var(model.BUS)
model.gen_supply = Var(model.GEN)
model.line_flow  = Var(model.BRANCH)

### Objective function
def objfunction(model,i):
    totalcost = sum(model.gen_supply[i]*model.gen_C[i] for i in model.GEN) #* model.BaseMVA
    return totalcost

model.cost = Objective(rule=objfunction, sense=minimize)


### constraint
#Power balance equation
def PowerBal(model,k):
    buspower = sum(model.line_flow[j] for j in model.BRANCH if model.branch_tbus[j] == k) - sum(model.line_flow[j] for j in model.BRANCH if model.branch_fbus[j] ==k) + sum(model.gen_supply[i] for i in model.GEN if model.gen_bus[i]==k) - model.bus_Pd[k]
    return buspower == 0
model.cons_PowerBal = Constraint(model.BUS,rule=PowerBal)

#power output limitation for generator
def PGenMax(model,i):
    return model.gen_supply[i] <= model.gen_Pmax[i]
model.cons_PGenMax = Constraint(model.GEN,rule=PGenMax)
def PGenMin(model,i):
    return model.gen_Pmin[i] <=  model.gen_supply[i]
model.cons_PGenMin = Constraint(model.GEN,rule=PGenMin)


#line flow equation
def lineflow(model,j):
    return model.line_flow[j] == model.Base * ((model.bus_angle[model.branch_fbus[j]]-model.bus_angle[model.branch_tbus[j]])/model.branch_x[j])
model.cons_lineflow = Constraint(model.BRANCH, rule=lineflow)

#thermal constraint
def ThermalMax(model,j):
    return model.line_flow[j] <= model.branch_rateA[j]
model.cons_ThermalMax = Constraint(model.BRANCH, rule=ThermalMax)
def ThermalMin(model,j):
    return -model.branch_rateA[j] <=  model.line_flow[j] 
model.cons_ThermalMin = Constraint(model.BRANCH, rule=ThermalMin)



In [5]:
solver = SolverFactory('solver\gurobi.exe')
samples = 10
dcopf = model.create_instance()
results = solver.solve(dcopf)

    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Gurobi 9.5.1\x3a infeasible; constraint.dunbdd
        returned.; 1426 simplex iterations


In [6]:
print(results)


Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 0
  Number of variables: 280
  Sense: unknown
Solver: 
- Status: warning
  Message: Gurobi 9.5.1\x3a infeasible; constraint.dunbdd returned.; 1426 simplex iterations
  Termination condition: infeasible
  Id: 203
  Error rc: 0
  Time: 0.07812118530273438
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [7]:
# form the pyomo data file
# form_data.formdata()
# instance according on the dat file

### set the solver
solver = SolverFactory('solver\gurobi.exe')
samples = 100
dcopf = model.create_instance()

Pd_List = np.zeros([samples,Bus_data.shape[0]])
Pg_List = np.zeros([samples,Gen_data.shape[0]])
PF_List = np.zeros([samples,Bra_data.shape[0]])
TC_List = np.zeros([samples,1])

### Suppress infeasible error warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

### Run solver to generate data samples
i = 0
while i < samples:
    load_profile = rand_load(bus,Bus_data)
    dcopf = model.create_instance()
    results = solver.solve(dcopf, tee=False)
    print(results)
    if results.solver.termination_condition == "optimal":        
        Pd_List[i,:] = load_profile
        Pg_List[i,:] = dcopf.gen_supply[:]()
        PF_List[i,:] = dcopf.line_flow[:]()
        TC_List[i,:] = dcopf.cost[:]()
        i = i+1
        if i % 100 == 0:
            print(i)
    

# ## Write results to CSV files    
# with open(("Pd_Profile_{}.csv".format(bus)), 'w', newline='') as csv_file:
#     csv_writer = csv.writer(csv_file, delimiter=',')
#     for i in range(Pd_List.shape[0]):
#         csv_writer.writerow(Pd_List[i,:])
# with open(("Pg_Profile_{}.csv".format(bus)), 'w', newline='') as csv_file:
#     csv_writer = csv.writer(csv_file, delimiter=',')
#     for i in range(Pg_List.shape[0]):
#         csv_writer.writerow(Pg_List[i,:])        
# with open(("PF_Profile_{}.csv".format(bus)), 'w', newline='') as csv_file:
#     csv_writer = csv.writer(csv_file, delimiter=',')
#     for i in range(PF_List.shape[0]):
#         csv_writer.writerow(PF_List[i,:])        
# with open(("TC_Profile_{}.csv".format(bus)), 'w', newline='') as csv_file:
#     csv_writer = csv.writer(csv_file, delimiter=',')
#     for i in range(TC_List.shape[0]):
#         csv_writer.writerow(TC_List[i,:])          
        


Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 0
  Number of variables: 280
  Sense: unknown
Solver: 
- Status: warning
  Message: Gurobi 9.5.1\x3a infeasible; constraint.dunbdd returned.; 496 simplex iterations
  Termination condition: infeasible
  Id: 203
  Error rc: 0
  Time: 0.04819822311401367
Solution: 
- number of solutions: 0
  number of solutions displayed: 0


Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 0
  Number of variables: 280
  Sense: unknown
Solver: 
- Status: warning
  Message: Gurobi 9.5.1\x3a infeasible; constraint.dunbdd returned.; 474 simplex iterations
  Termination condition: infeasible
  Id: 203
  Error rc: 0
  Time: 0.05306220054626465
Solution: 
- number of solutions: 0
  number of solutions displayed: 0


Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 595
  Number of variables: 280
  Sense: unknown